In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import datetime

#  Import and read the csv.
import pandas as pd 
waste_df = pd.read_csv("./resources/austin_waste_and_diversion.csv", error_bad_lines=False, delimiter=',', skip_blank_lines=True)
waste_df.head()


,dropoff_site,load_id,load_time,load_type,load_weight,report_date,route_number,route_type
0,MRF,328118,5/27/2009 12:55,RECYCLING - SINGLE STREAM,4060.0,5/27/2009,DW1,RECYCLING - SINGLE STREAM
1,BRAKER SITE,308336,12/16/2008 10:55,SWEEPING,NaN,12/16/2008,NW10-R,STREET CLEANING-RESIDENTIAL
2,TDS - MRF,541514,9/6/2012 13:56,RECYCLING - SINGLE STREAM,5060.0,9/6/2012,RHBU20,RECYCLING - SINGLE STREAM
3,TDS LANDFILL,689059,3/14/2016 7:56,SWEEPING,3720.0,3/14/2016,DSS04,SWEEPER DUMPSITES
4,HORNSBY BEND,108272,12/2/2004 15:29,YARD TRIMMING,13840.0,12/2/2004,YH08,YARD TRIMMINGS


In [2]:
routes_df = pd.read_csv("./resources/GarbageRecycle2015RR.csv", error_bad_lines=False, delimiter=',', skip_blank_lines=True)
routes_df.head()

,LANDFILL,the_geom,GARB_RT,GARB_DAY,GARB_SUP,SUPER_NUM,OP_TYPE,RT_OLD
0,TDS,MULTIPOLYGON (((-97.70618263474258 30.36741600...,PAH83,Thursday,Gilbert,697.0,Auto,NaN
1,TDS,MULTIPOLYGON (((-97.83236579358113 30.16212865...,PAM54,Monday,Castillo,500.0,Auto,PAM84
2,TDS,MULTIPOLYGON (((-97.72977104057442 30.19009265...,PAM71,Monday,Carnline,596.0,Auto,NaN
3,TDS,MULTIPOLYGON (((-97.65374103416144 30.26269210...,PAH54,Thursday,Castillo,500.0,Auto,PAH75
4,TDS,MULTIPOLYGON (((-97.64683377149026 30.38640132...,PAW71,Wednesday,Carnline,596.0,Auto,NaN


In [3]:
waste_df.dtypes

dropoff_site     object
load_id           int64
load_time        object
load_type        object
load_weight     float64
report_date      object
route_number     object
route_type       object
dtype: object

In [4]:
waste_df.describe()

,load_id,load_weight
count,535181.000000,4.788040e+05
mean,421736.006940,1.158039e+04
std,198963.512162,7.800556e+03
min,101222.000000,-4.480000e+03
25%,245010.000000,5.440000e+03
50%,387215.000000,1.082000e+04
75%,608054.000000,1.656000e+04
max,750483.000000,1.562821e+06


In [5]:
waste_df.nunique()

dropoff_site        33
load_id         535143
load_time       423013
load_type           18
load_weight       5116
report_date       4342
route_number      1787
route_type          37
dtype: int64

In [6]:
route_total = waste_df.groupby(["route_number"]).count()["route_type"]
route_total

route_number
0BM00            10
ABR01             3
AFD-FIREWISE      9
BKR-BR            9
BKR-YT           74
               ... 
YW12            223
YW13             59
YWNON1           28
YWNON5            3
ZILKER           51
Name: route_type, Length: 1787, dtype: int64

In [7]:
load_counts = waste_df.groupby(["load_type"]).count()["load_id"]
load_counts

load_type
BAGGED LITTER                      40
BRUSH                           30043
BULK                            29373
DEAD ANIMAL                      5109
GARBAGE COLLECTIONS            188255
LITTER                           1187
MATTRESS                            3
MIXED LITTER                     1223
MULCH                            1276
RECYCLED METAL                    613
RECYCLING - COMINGLE            29533
RECYCLING - PAPER               30490
RECYCLING - PLASTIC BAGS           38
RECYCLING - SINGLE STREAM       94336
SWEEPING                        68599
TIRES                            2237
YARD TRIMMING                   52814
YARD TRIMMING - X-MAS TREES        12
Name: load_id, dtype: int64

In [8]:
# Convert load_time to datetime 
waste_df["load_time"] = pd.to_datetime(waste_df["load_time"])
waste_df.dtypes

dropoff_site            object
load_id                  int64
load_time       datetime64[ns]
load_type               object
load_weight            float64
report_date             object
route_number            object
route_type              object
dtype: object

In [15]:
# Extract year from load_time to new load_year column
waste_df["load_year"] = waste_df["load_time"].apply(lambda x: x.year)
waste_df.head(5)

,dropoff_site,load_id,load_time,load_type,load_weight,report_date,route_number,route_type,load_year
0,MRF,328118,2009-05-27 12:55:00,RECYCLING - SINGLE STREAM,4060.0,5/27/2009,DW1,RECYCLING - SINGLE STREAM,2009
1,BRAKER SITE,308336,2008-12-16 10:55:00,SWEEPING,NaN,12/16/2008,NW10-R,STREET CLEANING-RESIDENTIAL,2008
2,TDS - MRF,541514,2012-09-06 13:56:00,RECYCLING - SINGLE STREAM,5060.0,9/6/2012,RHBU20,RECYCLING - SINGLE STREAM,2012
3,TDS LANDFILL,689059,2016-03-14 07:56:00,SWEEPING,3720.0,3/14/2016,DSS04,SWEEPER DUMPSITES,2016
4,HORNSBY BEND,108272,2004-12-02 15:29:00,YARD TRIMMING,13840.0,12/2/2004,YH08,YARD TRIMMINGS,2004


In [16]:
# Filter by 2017
df_2017 = waste_df[waste_df.load_year == 2017]
df_2017.head()

,dropoff_site,load_id,load_time,load_type,load_weight,report_date,route_number,route_type,load_year
10,TDS LANDFILL,732809,2017-03-22 07:40:00,GARBAGE COLLECTIONS,15840.0,3/22/2017,SXSW2017,SPECIAL EVENTS,2017
56,TDS LANDFILL,736509,2017-04-19 17:35:00,GARBAGE COLLECTIONS,20680.0,4/19/2017,PAW50,GARBAGE COLLECTION,2017
62,TDS LANDFILL,724256,2017-01-12 13:29:00,GARBAGE COLLECTIONS,21660.0,1/9/2017,PM41,GARBAGE COLLECTION,2017
90,HORNSBY BEND,747204,2017-07-20 12:36:00,YARD TRIMMING,5720.0,7/20/2017,YH12,YARD TRIMMINGS,2017
92,TDS LANDFILL,740763,2017-05-24 00:00:00,BULK,5580.0,5/24/2017,BU07,BULK,2017


In [17]:
df_2017.nunique()

dropoff_site       20
load_id         26045
load_time       18951
load_type          16
load_weight      2037
report_date       253
route_number      977
route_type         26
load_year           1
dtype: int64

In [18]:
df_2017 = df_2017.drop(["load_id","load_time"], axis = 1)
df_2017.head()

,dropoff_site,load_type,load_weight,report_date,route_number,route_type,load_year
10,TDS LANDFILL,GARBAGE COLLECTIONS,15840.0,3/22/2017,SXSW2017,SPECIAL EVENTS,2017
56,TDS LANDFILL,GARBAGE COLLECTIONS,20680.0,4/19/2017,PAW50,GARBAGE COLLECTION,2017
62,TDS LANDFILL,GARBAGE COLLECTIONS,21660.0,1/9/2017,PM41,GARBAGE COLLECTION,2017
90,HORNSBY BEND,YARD TRIMMING,5720.0,7/20/2017,YH12,YARD TRIMMINGS,2017
92,TDS LANDFILL,BULK,5580.0,5/24/2017,BU07,BULK,2017


In [22]:
# weight of trash by year by route
df_2017_new = df_2017.groupby(["route_number"]).sum(["load_weight"])
df_2017_new = pd.DataFrame(df_2017_new)
df_2017_new = df_2017_new.drop(["load_year"], axis=1)
df_2017_new.head(10)

,load_weight
route_number,
0BM00,117420.0
BLN-01,0.0
BR01,525100.0
BR02,388520.0
BR03,346910.0
BR04,213920.0
BR05,719665.0
BR06,382520.0
BR07,409180.0


In [ ]:
#df_2017_new = df_2017_new.loc[df_2017["load_weight"] == 0]

In [23]:
routes_df.head()

,LANDFILL,the_geom,GARB_RT,GARB_DAY,GARB_SUP,SUPER_NUM,OP_TYPE,RT_OLD
0,TDS,MULTIPOLYGON (((-97.70618263474258 30.36741600...,PAH83,Thursday,Gilbert,697.0,Auto,NaN
1,TDS,MULTIPOLYGON (((-97.83236579358113 30.16212865...,PAM54,Monday,Castillo,500.0,Auto,PAM84
2,TDS,MULTIPOLYGON (((-97.72977104057442 30.19009265...,PAM71,Monday,Carnline,596.0,Auto,NaN
3,TDS,MULTIPOLYGON (((-97.65374103416144 30.26269210...,PAH54,Thursday,Castillo,500.0,Auto,PAH75
4,TDS,MULTIPOLYGON (((-97.64683377149026 30.38640132...,PAW71,Wednesday,Carnline,596.0,Auto,NaN


In [24]:
routes_df_new = routes_df.drop(["LANDFILL","GARB_DAY", "GARB_SUP", "SUPER_NUM", "OP_TYPE", "RT_OLD"], axis= 1)
routes_df_new.tail()

,the_geom,GARB_RT
179,MULTIPOLYGON (((-97.77237758904216 30.42129789...,PAW63
180,MULTIPOLYGON (((-97.77342974546856 30.14454642...,PAM72
181,MULTIPOLYGON (((-97.77932023649187 30.15112227...,PAM73
182,MULTIPOLYGON (((-97.79985649924181 30.24749360...,PAT84
183,MULTIPOLYGON (((-97.60620180891306 30.38151623...,PAW70


In [ ]:
#routes_df_new.rename(columns={"GARB_RT" : "route_number"})
#routes_df_new.head()

In [26]:
routes_df_new.set_index(["GARB_RT"], inplace=True)
routes_df_new.head()

,the_geom
GARB_RT,
PAH83,MULTIPOLYGON (((-97.70618263474258 30.36741600...
PAM54,MULTIPOLYGON (((-97.83236579358113 30.16212865...
PAM71,MULTIPOLYGON (((-97.72977104057442 30.19009265...
PAH54,MULTIPOLYGON (((-97.65374103416144 30.26269210...
PAW71,MULTIPOLYGON (((-97.64683377149026 30.38640132...


In [27]:
# merge 
df_2017_routes = df_2017_new.merge(routes_df_new, how="left", left_index=True, right_index=True)
df_2017_routes.head()

,load_weight,the_geom
route_number,,
0BM00,117420.0,NaN
BLN-01,0.0,NaN
BR01,525100.0,NaN
BR02,388520.0,NaN
BR03,346910.0,NaN


In [29]:
#Drop na
df_2017_routes = df_2017_routes.dropna()
df_2017_routes.head()

,load_weight,the_geom
route_number,,
PAF01,632960.0,MULTIPOLYGON (((-97.80021375787697 30.36733765...
PAF02,678100.0,MULTIPOLYGON (((-97.77221898831849 30.32271999...
PAF03,649520.0,MULTIPOLYGON (((-97.7601229300713 30.340361143...
PAF04,651640.0,MULTIPOLYGON (((-97.77662750513846 30.30588241...
PAF50,545080.0,MULTIPOLYGON (((-97.7724542400386 30.297463218...


In [30]:
#Description
df_2017_routes.describe()

,load_weight
count,1.840000e+02
mean,7.379733e+05
std,1.783667e+05
min,3.498200e+05
25%,6.325300e+05
50%,7.120500e+05
75%,8.259200e+05
max,1.325600e+06


In [31]:
df_2017_routes.to_csv("routes_2017_weight.csv")